追従実験考えてみる
1. ユーザーをsimhashでいい感じにコホートに分解できるのか
2. コホートの性質の分析
3. コホートからリバースエンジニアリングできるか？

1. ユーザーをsimashでいい感じにコホートに分解できるのか  
シンプルなoneHotで試してみる．

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
d = 1000
n = 10000
root_cohort_num = 10 # 大カテゴリ， Web上の履歴に大きく影響を及ぼす(性別とか年齢とか)
p_root_cohort = 0.01
week_cohort_num = 100 # 少カテゴリ， 小さい確率でそれっぽい行動をする(個人端末で仕事のこと調べるとか)
p_week_cohort = 0.005
p_noize = 0.001

In [144]:
cohort_roots = [np.random.binomial(10, p_root_cohort, d) for i in range(root_cohort_num)]
cohort_week = [np.random.binomial(10, p_week_cohort, d) for i in range(week_cohort_num)]
result = []
for i in range(0, n):
  # 誤差乗せる
  root_idx = i % root_cohort_num
  week_idx = i % week_cohort_num
  result.append(np.where(cohort_roots[root_idx] + cohort_week[week_idx] + np.random.binomial(5, p_noize, d), 1, 0) )
  #誤差 ０.01だとそこそこブレる， まぁぶれたならそこが大事だからちゃんと反映されるねという話はある
  #よく考えたらカラムはドメインなので， 0.01 * dした数だけ違うドメインに飛んでたらそれは違うクラスタだわ

In [145]:
weights = [np.random.randint(-1, 2, d) for i in range(100)] # 適当に100bitでやってみる

In [146]:
vecs = []
for i in range(n):
  vec = []
  for weight in weights:
    vec.append(1 if np.dot(result[i], weight) >= 0 else 0)
  vecs.append(vec)

In [147]:
bites = [bidnizer(each) for each in vecs]

In [148]:
def bidnizer(tgt : list):
  out = 0
  for each in tgt:
    out = (out << 1) | each
  return out

In [149]:
cohort_id = []
week_cohort_id = []
for i in range(0, n):
  root_idx = i % root_cohort_num
  week_idx = i % week_cohort_num
  cohort_id.append(root_idx)
  week_cohort_id.append(week_idx)

In [150]:
tmpDf = pd.DataFrame(list(zip(bites, cohort_id, week_cohort_id)), columns = ["bite", "cohort_id", "week_cohort_id"])

In [151]:
sTmpDf = tmpDf.sort_values("bite")

In [152]:
pd.set_option("max_rows", 100000)

In [153]:
sTmpDf
# cohort_idの並びを見るとそこそこできている…のか？
# 弱いコホートに関してもまぁある程度集約できてる
# ノイズによって変わるけど十分大きくするとメモリが足りなくなって面倒なのでココで終わる

,bite,cohort_id,week_cohort_id
306,643095271761254918970874702,6,6
906,645512537715796887090699086,6,6
8606,647969215284293144475278158,6,6
4006,652803940741341578858273614,6,6
5006,657639459552359358459485006,6,6
2506,1881035163472260387469466446,6,6
2006,1890703618694100077892541278,6,6
8906,1890705979733367062125550414,6,6
1906,1890706127307319651804060494,6,6
7406,1895541904373321811341024078,6,6


In [ ]:
sTmpDf.loc[(sTmpDf.cohort_id == 69) & (sTmpDf.week_cohort_id == 869)]

,bite,cohort_id,week_cohort_id
869,17681930828797471918457330534,69,869
4869,17759594865677998392051997028,69,869
1869,18300900848440179648093985126,69,869
7869,18301203079967123306164754790,69,869
5869,18301203079967140899425589604,69,869
6869,18306038783245581961376274788,69,869
9869,335213853137024473817779509604,69,869
3869,335218688840239952291259453798,69,869
2869,335223524543581390851177159012,69,869
8869,374837605798409968285004044646,69,869


KeyError: ignored